In [ ]:
import requests      # Для запросов по API
import json          # Для обработки полученных результатов
import time          # Для задержки между запросами
import os            # Для работы с файлами
import pandas as pd  # Для формирования датафрейма с результатами

In [ ]:
def getPage(page):
    params = {
        'employer_id': 39305,  # ID 2ГИС
        'page': page,         # Номер страницы
        'per_page': 100       # Кол-во вакансий на 1 странице
    }   
    req = requests.get('https://api.hh.ru/vacancies', params)
    data = req.content.decode() 
    req.close()
    return data
    
json_s = [] 

for page in range(0, 100):
    print(page)
    jsObj = json.loads(getPage(page))
    
    json_s+=jsObj['items']

    if (jsObj['pages'] - page) <= 1:  
        break
    time.sleep(0.2)


repr= []
for i in json_s:
    try:
        d_mvf={'id':str(i['id']),
        'name':i['name'],
        'department':i['department'] if i['department']==None else i['department']['name'],
        'response_letter_required':1 if i['response_letter_required'] else 0,
        'type':i['type']['name'],
        'created_at':i['created_at'][:-6],
        'employer': i['employer']['name'],
        'accredited_it_employer': 1 if 'accredited_it_employer' in i['employer'].keys() else 0,
        'schedule':i['schedule']['name'],
        'accept_temporary':1 if i['accept_temporary'] else 0,
        'professional_roles':i['professional_roles'][0]['name'],
        'experience':sum([int(i) for i in i['experience']['name'].split(' ') if i.isnumeric()])/2,
        'employment':i['employment']['name']}
        
        if 'salary' in i.keys():
            d_mvf['salary']=i['salary'] if i['salary']==None else i['salary']['from']
        repr+=[d_mvf]
    except:
        pass
repr = pd.DataFrame(repr)
repr

In [ ]:
for i in range(len(repr)):
    data= list(repr.iloc[i])
    print(data)
    cursor.execute('''INSERT INTO resume_data (
    id,name,department,response_letter_required,type,created_at,employer,accredited_it_employer,schedule,accept_temporary,professional_roles,experience,employment,salary

    ) VALUES (?, ?, ?, ?, ?,?, ?, ?, ?, ?, ?, ?, ?, ?)''', data)
    connection.commit()

In [1]:
import sqlite3
import os
# Устанавливаем соединение с базой данных

dir_current = os.getcwd()
# Устанавливаем соединение с базой данных
connection = sqlite3.connect(dir_current+'/my_database.db')
cursor = connection.cursor()

cursor.execute('''drop TABLE resume_data ''')
# Создаем таблицу Users
cursor.execute('''CREATE TABLE IF NOT EXISTS resume_data (
id                           text,
name                         text,
department                   text,
response_letter_required       text,
type                         text,
created_at                   date,
employer                     text,
accredited_it_employer         text,
schedule                     text,
accept_temporary               text,
professional_roles           text,
experience                  float,
employment                   text,
salary                      float

)''')

In [ ]:
import os
